In [1]:
import pandas as pd
import numpy as np
import torch

# 3. `Human_Numbers`

`HUMAN_NUMBERS_train.txt` 사람이 읽을 수 있는 형식으로 숫자가 나열된
텍스트 파일이다. 이 텍스트 파일을 이용하여 “현재 단어가 주어졌을 때 다음
단어를 예측하는” 신경망을 설계하고 학습시킬 것이다. 아래는 해당 파일을
불러와 단어 단위로 나눈 결과를 확인하는 코드이다.

In [2]:
with open('HUMAN_NUMBERS_train.txt') as f:
    words = f.read().split()
print(words[-12:])

['seven', 'thousand', 'nine', 'hundred', 'ninety', 'eight', 'seven', 'thousand', 'nine', 'hundred', 'ninety', 'nine']


이 텍스트 파일에는 예를 들어 “… seven, thousand, nine, hundred, ninety,
eight, …” 등과 같은 사람이 읽는 숫자 표현이 단어 단위로 나열된 시퀀스
형태로 저장되어 있다.

이러한 데이터에서 “현재 단어가 주어졌을 때 다음 단어를 예측하는” 자연어
처리 모델을 학습하라. 즉 아래와 같은 맵핑을 학습하라.

-   … seven, thousand, nine, hundred, ninety, eight $\to$ seven
-   … seven, thousand, nine, hundred, ninety, eight, seven $\to$
    thousand
-   … seven, thousand, nine, hundred, ninety, eight, seven, thousand
    $\to$ nine

**제약사항**

-   one-hot 전처리 코드를 포함할 것
-   `torch.nn.RNN`, `torch.nn.RNNCell`, `torch.nn.LSTM` 중 하나를 이용할
    것
-   처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을
    제시할 것

`(풀이)`

`-` 데이터 불러오기 및 전처리

In [3]:
with open('HUMAN_NUMBERS_train.txt') as f:
    words = f.read().split()
words[:10]
df_train = pd.DataFrame({'x': words[:-1], 'y': words[1:]})

# 전체 vocab 기준으로 맵핑
vocab = sorted(set(words))
dct= {w: i for i, w in enumerate(vocab)}
# train 데이터셋
x = torch.tensor(df_train.x.map(dct))
y = torch.tensor(df_train.y.map(dct))

In [4]:
# one-hot 전처리
X = torch.nn.functional.one_hot(x).float()
y = torch.nn.functional.one_hot(y).float()

In [5]:
with open('HUMAN_NUMBERS_valid.txt') as f:
    words = f.read().split()
words[:10]
df_valid = pd.DataFrame({'x': words[:-1], 'y': words[1:]})

# valid 데이터셋
xx = torch.tensor(df_valid.x.map(dct))
yy = torch.tensor(df_valid.y.map(dct))

# one-hot 전처리
XX = torch.nn.functional.one_hot(xx).float()
yy = torch.nn.functional.one_hot(yy).float()

`-` 모형 적합

In [7]:
lstm = torch.nn.LSTM(29, 64)
cook = torch.nn.Sequential(
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 29)
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(lstm.parameters()) + list(cook.parameters()))

#---#
for epoc in range(1, 2001) :
    cook.train()
    h, _ = lstm(X)
    netout = cook(h)

    loss = loss_fn(netout, y)
    loss.backward()

    optimizr.step()
    optimizr.zero_grad()

    if epoc % 100 == 0 :
        cook.eval()
        h, _ = lstm(X)
        hh, _ = lstm(XX)
        yhat = torch.nn.functional.softmax(cook(h), dim = 1)
        yyhat = torch.nn.functional.softmax(cook(hh), dim = 1)
        acc = (yhat.argmax(axis = 1) == y.argmax(axis = 1)).float().mean().item()
        valid_acc = (yyhat.argmax(axis = 1) == yy.argmax(axis = 1)).float().mean().item()
        print(f"epoch : {epoc}\ttrain acc = {acc:.4f}\tvalid acc = {valid_acc:.4f}")

        if acc > 0.992 :
            break

epoch : 100	train acc = 0.1711	valid acc = 0.1636
epoch : 200	train acc = 0.3613	valid acc = 0.3514
epoch : 300	train acc = 0.3979	valid acc = 0.3623
epoch : 400	train acc = 0.4507	valid acc = 0.4012
epoch : 500	train acc = 0.5112	valid acc = 0.4119
epoch : 600	train acc = 0.6222	valid acc = 0.4485
epoch : 700	train acc = 0.7563	valid acc = 0.4909
epoch : 800	train acc = 0.8980	valid acc = 0.5305
epoch : 900	train acc = 0.9602	valid acc = 0.5478
epoch : 1000	train acc = 0.9810	valid acc = 0.5657
epoch : 1100	train acc = 0.9892	valid acc = 0.5855
epoch : 1200	train acc = 0.9929	valid acc = 0.5936


In [8]:
for epoc in range(1201, 3001) :
    cook.train()
    h, _ = lstm(X)
    netout = cook(h)

    loss = loss_fn(netout, y)
    loss.backward()

    optimizr.step()
    optimizr.zero_grad()

    if epoc % 100 == 0 :
        cook.eval()
        h, _ = lstm(X)
        hh, _ = lstm(XX)
        yhat = torch.nn.functional.softmax(cook(h), dim = 1)
        yyhat = torch.nn.functional.softmax(cook(hh), dim = 1)
        acc = (yhat.argmax(axis = 1) == y.argmax(axis = 1)).float().mean().item()
        valid_acc = (yyhat.argmax(axis = 1) == yy.argmax(axis = 1)).float().mean().item()
        print(f"epoch : {epoc}\ttrain acc = {acc:.4f}\tvalid acc = {valid_acc:.4f}")

        if acc > 0.9999 :
            break

epoch : 1300	train acc = 0.9947	valid acc = 0.6014
epoch : 1400	train acc = 0.9962	valid acc = 0.6050
epoch : 1500	train acc = 0.9971	valid acc = 0.6069
epoch : 1600	train acc = 0.7265	valid acc = 0.5282
epoch : 1700	train acc = 0.9283	valid acc = 0.6413
epoch : 1800	train acc = 0.9725	valid acc = 0.6563
epoch : 1900	train acc = 0.9856	valid acc = 0.6564
epoch : 2000	train acc = 0.9903	valid acc = 0.6554
epoch : 2100	train acc = 0.9926	valid acc = 0.6544
epoch : 2200	train acc = 0.9941	valid acc = 0.6565
epoch : 2300	train acc = 0.9954	valid acc = 0.6559
epoch : 2400	train acc = 0.9961	valid acc = 0.6561
epoch : 2500	train acc = 0.9968	valid acc = 0.6556
epoch : 2600	train acc = 0.9976	valid acc = 0.6559
epoch : 2700	train acc = 0.9981	valid acc = 0.6556
epoch : 2800	train acc = 0.9984	valid acc = 0.6564
epoch : 2900	train acc = 0.9986	valid acc = 0.6578
epoch : 3000	train acc = 0.9989	valid acc = 0.6586


`-` 결과 제시

In [9]:
inv_dct = {v:k for k, v in dct.items()}

print(
    f"처음 12개 단어 적합값 : {yhat.argmax(axis = 1)[:12]}\n"
    f"인덱스별 단어 변환 : {[inv_dct[t.item()] for t in yhat.argmax(axis = 1)[:12]]}\n"
    "---------------------------------------------------------------------------------\n"
    f"마지막 12개 단어 적합값 : {yhat.argmax(axis = 1)[-12:]}\n"
    f"인덱스별 단어 변환 : {[inv_dct[t.item()] for t in yhat.argmax(axis = 1)[-12:]]}"
)

처음 12개 단어 적합값 : tensor([ 6,  6, 18, 18, 18, 14, 27, 11,  6,  3, 26, 22])
인덱스별 단어 변환 : ['five', 'five', 'six', 'six', 'six', 'one', 'twenty', 'nine', 'five', 'eleven', 'twelve', 'thirteen']
---------------------------------------------------------------------------------
마지막 12개 단어 적합값 : tensor([15, 24, 11, 10, 13,  0, 15, 24, 11, 10, 13, 11])
인덱스별 단어 변환 : ['seven', 'thousand', 'nine', 'hundred', 'ninety', 'eight', 'seven', 'thousand', 'nine', 'hundred', 'ninety', 'nine']
